<a href="https://colab.research.google.com/github/Nikitapalakodeti/Generative-AI-projects/blob/main/ResearchAssist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.284
!pip install python-dotenv==1.0.0
!pip install streamlit==1.22.0
!pip install unstructured==0.9.2
!pip install tiktoken==0.4.0
!pip install faiss-cpu==1.7.4
!pip install libmagic==1.0
!pip install python-magic==0.4.27
!pip install python-magic-bin==0.4.14
!pip install openai==0.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.1 which 

In [ ]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader

**1)LOAD DATA**

In [ ]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.sumobrain.com/patents/wipo/Rational-identification-fda-approved-drugs/WO2022043816A2.html"
])
data = loaders.load()
len(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


1

**2)SPLIT THE DATA INTO CHUNKS**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

61

In [ ]:
docs[0].page_content

'Login|\n\nSign Up|\n\nHelp|\n\nContact|\n\nHome\xa0\xa0|\n\nSearch\xa0\xa0|\n\nSearch Tutorial\xa0\xa0|\n\nTitle:\n\nRATIONAL IDENTIFICATION OF FDA APPROVED DRUGS BY COMPUTATIONAL MODELLING AND INHIBITION OF RBD/ACE2 BINDING FOR TREATMENT OF COVID-19\n\nDocument Type and Number:\n\nWIPO Patent Application WO/2022/043816\n\nKind Code:\n\nA2\n\nAbstract:\n\nThe present invention provides 3D bioprinted models for COVID-19 that simulates endothelial dysfunction, viral entry and other associated inflammatory responses caused by the virus and thus can be used for discovering new drugs, repurposing existing drugs used for other indication, testing monoclonal and polyclonal antibodies/passive vaccines with neutralizing properties and other nutraceuticals, herbal products for COVID-19. Using the platform technology of 3D bioprinting applicants have identified Ertugliflozin as a novel candidate for immediate repurposing as a treatment option for COVID-19 infections.\n\nInventors:\n\n\n         

In [ ]:
documents = [doc.page_content for doc in docs]

**3)INSERTING MODEL**

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Nuclear fission occurs when a heavy atom, such as uranium-238 or plutonium-239, is split in two to form two light atomic nuclei: two less massive sub-atomic particles. During fission, each of the resulting nuclei typically contains some additional energy, which can be converted to thermal energy to yield heat, light, or glow.

Nuclear fusion, also called nuclear fission, occurs when two or more nuclei combine to form one or more new nuclei. When two or more nuclei unite, the combined mass is more than the sum of the masses of each nucleus, and the new mass may contain more energy than the sum of the masses of the two original nuclei. The energy released when nuclei fuse is primarily in the form of heat. Nuclear fusion requires temperatures exceeding 100 million degrees Celsius, while nuclear fission occurs at lower temperatures.


**4)CREATING EMBEDDINGS**

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.5 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.02250259 -0.07829171 -0.02303071 ... -0.0082793   0.02652686
  -0.00201896]
 [ 0.04170233  0.00109741 -0.01553419 ... -0.02181627 -0.06359359
  -0.00875289]]


**5)STORING INTO VECTOR DATABASE**

In [ ]:
import pickle
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pandas as pd


class VectorStore:
    def __init__(self, chunks: list[str], embeddings_path: str = 'embeddings.pkl', model=None,
                 N: int = 5, metric: str = 'cosine'):
        print('vector store initializing ...')
        self._path: str = embeddings_path
        self._model=model
        self._embeddings: list[dict] or None = (
            self._embed_texts(chunks, False) if model is not None
            else self._load()
            )
        self._metric = metric
        self._n = N

    def get_path(self) -> str:
        return self._path

    def get_embeddings(self) -> list[dict]:
        return self._embeddings

    def _load(self) -> list[list[float]]:
        try:
            return pickle.load(self.get_path())

        except Exception as e:
            print(e)
            print('something went wrong while loading embeddings')
            return []

    def _embed_texts(self, texts: list[str],
                     is_question: bool = False) -> list[dict]:
        try:
            _embeddings = self._model.encode(texts)
            print('embeddings created successfully..')

            if not is_question:
                embeddings: list[dict] = [
                    {'text': txt, 'embeddings': embedding}
                    for txt, embedding in zip(texts, _embeddings.tolist())
                    ]
                self._dump_embeddings(embeddings)

            else:
                embeddings = _embeddings.tolist()

            return embeddings

        except Exception as e:
            print(e)
            print('something went wrong while embedding the texts')
            return []

    @staticmethod
    def _dump_embeddings(embeddings):
        try:
            pickle.dump(embeddings, open('embeddings.pkl', 'wb'))
            print('embeddings stored successfully.')

        except Exception as e:
            print(e)
            print('something went wrong while writing embeddings...')

    def _get_top_n_chunks_cosine(self, question_embeddings: list[float]) -> list[str]:
        top_n_chunks: list[dict] = []
        print(f'fetching top {self._n} relevant paragraphs')

        try:
            for chunk in tqdm(self._embeddings):
                chunk_text: str = chunk.get('text')
                chunk_embeddings: list[float] = chunk.get('embeddings')
                _score = cosine_similarity([chunk_embeddings,
                                                   question_embeddings])
                top_n_chunks.append({
                    'text': chunk_text,
                    'score': _score[0][1]
                })

            _chunks: list[str] = pd.DataFrame(top_n_chunks).sort_values(by='score', ascending=False)['text'].to_list()
            return _chunks[:self._n] if self._n <= len(_chunks) else _chunks

        except Exception as e:
            print(e)
            print('something went wrong while fetching the top paragraphs based on cosine similarity')

    def _get_top_n_chunks(self, question_embeddings: list[float]) -> list[str]:
        if self._metric == 'cosine':
            return self._get_top_n_chunks_cosine(question_embeddings)

        else:
            print(f'invalid metric {self._metric}')
            return []

    def get_relevant_chunks(self, question: str) -> list[str]:
        question_embeddings = self._embed_texts(texts=[question],
                                                is_question=True)
        question_embeddings = question_embeddings[0] if isinstance(question_embeddings, list) and isinstance(question_embeddings[0], list) and question_embeddings[0] else []

        if question_embeddings:
            return self._get_top_n_chunks(question_embeddings)
        else:
            print('something went wrong while fetching embeddings of the question.')
            return []


vs = VectorStore(documents, 'embeddings.pkl', SentenceTransformer('sentence-transformers/all-mpnet-base-v2'))
question="identify repurposed drug for covid 19"
relevant_chunks = vs.get_relevant_chunks(question)
print(relevant_chunks)

vector store initializing ...
embeddings created successfully..
embeddings stored successfully.
embeddings created successfully..
fetching top 5 relevant paragraphs


100%|██████████| 61/61 [00:00<00:00, 2132.55it/s]

['Drug repurposing refers to the identification of novel applications for an approved or investigational/experimental drug outside the premise of its medical indication. At present, this strategy would be a logical choice for developing a new drug for COVID-19 considering the substantial time-scales associated with new drug discovery and the trial-based validation of its safety and efficacy. The major advantage of repurposed drug is that it has been already evaluated for safety in human trials, which would reduce the significant amounts of time and money, a priority concern in SARS-CoV-2 drug development.', 'BACKGROUND OF THE INVENTION\n\nMitigation of COVID-19 prophylactical ly and therapeutically has been proposed and are currently in practise. In early 2020, many anti-viral compounds were proposed (WHO Solidarity trial results, 2021), natural products were reported (Natural and Nature-Derived Products Targeting Human Coronaviruses. 2021, Molecules, 26(2), 448) and now, vaccines were

**6)FETCHING RELEVANT CHUNKS**

In [ ]:
print(relevant_chunks)

['Drug repurposing refers to the identification of novel applications for an approved or investigational/experimental drug outside the premise of its medical indication. At present, this strategy would be a logical choice for developing a new drug for COVID-19 considering the substantial time-scales associated with new drug discovery and the trial-based validation of its safety and efficacy. The major advantage of repurposed drug is that it has been already evaluated for safety in human trials, which would reduce the significant amounts of time and money, a priority concern in SARS-CoV-2 drug development.', 'BACKGROUND OF THE INVENTION\n\nMitigation of COVID-19 prophylactical ly and therapeutically has been proposed and are currently in practise. In early 2020, many anti-viral compounds were proposed (WHO Solidarity trial results, 2021), natural products were reported (Natural and Nature-Derived Products Targeting Human Coronaviruses. 2021, Molecules, 26(2), 448) and now, vaccines were

In [ ]:
len(relevant_chunks)

5

In [ ]:
chunks_text = '\n\n'.join(relevant_chunks)

**7)QUESTION AND ANSWER PROMPT**

In [ ]:
qna_prompt='''# OBJECTIVE #
To answer a given "Question" based ON ONLY THE "Reference Material" provided in an informative and precise manner. Don't assume anything, in case the "Reference Material" are not helpful in answering the "Question", ask for further details from the user.

>>

# Reference Material #
{CHUNKS}

>>

# INSTRUCTIONS #
- When you feel the Reference Material have content related to the question, generate the answer like how a Chatbot would respond to the question. This is to ensure the user feels they are interacting with another human.
>>

# Question #
{QUESTION}'''.replace('{CHUNKS}',chunks_text).replace('{QUESTION}',question)

In [ ]:
print(qna_prompt)

# OBJECTIVE #
To answer a given "Question" based ON ONLY THE "Reference Material" provided in an informative and precise manner. Don't assume anything, in case the "Reference Material" are not helpful in answering the "Question", ask for further details from the user.

>>

# Reference Material #
Drug repurposing refers to the identification of novel applications for an approved or investigational/experimental drug outside the premise of its medical indication. At present, this strategy would be a logical choice for developing a new drug for COVID-19 considering the substantial time-scales associated with new drug discovery and the trial-based validation of its safety and efficacy. The major advantage of repurposed drug is that it has been already evaluated for safety in human trials, which would reduce the significant amounts of time and money, a priority concern in SARS-CoV-2 drug development.

BACKGROUND OF THE INVENTION

Mitigation of COVID-19 prophylactical ly and therapeutically h

**8)OUTPUT**

In [ ]:
print(generate_text(qna_prompt))

[{'generated_text': 'Ertugliflozin has been shown to have anti-diabetic and anti-inflammatory activities and has also shown potential in a number of animal studies to inhibit the infection of SARS-CoV-2, the etiological agent of COVID-19'}]
